In [1]:
import math
import numpy as np
from torch import cos, sin
import scipy.optimize as opt
import torch
import torch.nn as nn
%matplotlib ipympl
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append("../../../ddn/")
from ddn.pytorch.node import *

from pytorch3d.loss import chamfer_distance
from pytorch3d.ops import sample_farthest_points
from descartes import PolygonPatch
from pytorch3d.io import IO, load_obj, save_obj,load_objs_as_meshes
from pytorch3d.structures import join_meshes_as_scene, Meshes, Pointclouds

from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)

from alpha_shapes import Alpha_Shaper, plot_alpha_shape

In [2]:
import faulthandler
faulthandler.enable()


In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cpu


In [4]:
torch.autograd.set_detect_anomaly(True)

def least_squares(u0, tgt_vtxs):
    """
    u0 are vertices
    """
    if not torch.is_tensor(u0):
        u0 = torch.tensor(u0)
    if not torch.is_tensor(tgt_vtxs):
        tgt_vtxs = torch.tensor(tgt_vtxs)

    res = torch.square(u0 - tgt_vtxs).sum()
    return res.double()

def least_squares_grad(u0, tgt_vtxs):
    if torch.is_tensor(u0):
        u0 = u0.detach().clone()
    else:
        u0 = torch.tensor(u0)
    if torch.is_tensor(tgt_vtxs):
        tgt_vtxs = tgt_vtxs.detach().clone()
    else:
        tgt_vtxs = torch.tensor(tgt_vtxs)
        
    # Ensure that u0 requires gradients
    gradient = 2 * (u0 - tgt_vtxs)
    return gradient.double()


def calculate_volume(vertices, faces):
    face_vertices = vertices[faces]  # (F, 3, 3)
    v0, v1, v2 = face_vertices[:, 0, :], face_vertices[:, 1, :], face_vertices[:, 2, :]
    
    # Compute determinant of the 3x3 matrix [v0, v1, v2]
    face_volumes = torch.det(torch.stack([v0, v1, v2], dim=-1)) / 6.0  # Shape: (F,)
    volume = face_volumes.sum()
    return volume.abs()


def volume_constraint(x, faces, tgt_vol):
    """
    Calculate the volume of a mesh using PyTorch tensors.
    Args:
        vertices_torch: Nx3 tensor of vertex coordinates
        faces: Mx3 array of face indices
    Returns:
        volume: Total volume of the mesh as a PyTorch scalar
    """
    if not torch.is_tensor(x):
        x = torch.tensor(x)
    if not torch.is_tensor(faces):
        faces = torch.tensor(faces)
    if not torch.is_tensor(tgt_vol):
        tgt_vol = torch.tensor(tgt_vol)

    vertices = x.view(-1,3)
    faces = faces.view(-1,3)    
    volume = calculate_volume(vertices, faces)
    res = volume.abs() - tgt_vol
    return res.double()

def volume_constraint_grad(x, faces):
    if torch.is_tensor(x):
        x = x.detach().clone()
    else:
        x = torch.tensor(x)
    if torch.is_tensor(faces):
        faces = faces.detach().clone()
    else:
        faces = torch.tensor(faces)

    vertices_torch = x.view(-1, 3)
    p0 = vertices_torch[faces[:, 0]]  # (F, 3)
    p1 = vertices_torch[faces[:, 1]]  # (F, 3)
    p2 = vertices_torch[faces[:, 2]]  # (F, 3)

    grad_p0 = torch.cross(p1, p2, dim=1) / 6.0
    grad_p1 = torch.cross(p2, p0, dim=1) / 6.0
    grad_p2 = torch.cross(p0, p1, dim=1) / 6.0

    grad_verts = torch.zeros_like(vertices_torch)
    grad_verts.scatter_add_(0, faces[:, 0].unsqueeze(1).expand(-1, 3), grad_p0)
    grad_verts.scatter_add_(0, faces[:, 1].unsqueeze(1).expand(-1, 3), grad_p1)
    grad_verts.scatter_add_(0, faces[:, 2].unsqueeze(1).expand(-1, 3), grad_p2)

    analytical_grad = grad_verts.flatten()
    return analytical_grad 


class ConstrainedProjectionNode(EqConstDeclarativeNode):

    def __init__(self, src: Meshes, tgt: Meshes):
        super().__init__(eps=1.0e-6) # relax tolerance on optimality test 
        self.src = src # source meshes (B,)
        self.tgt = tgt # target meshes (B,)

    def objective(self, xs: torch.Tensor, y: torch.Tensor, scatter_add=True):
        """
        Calculates sum of squared differences between source and target meshes.

        Args:
            xs (tensor): vertices of original mesh, sum(V_i) x 3
            y (tensor): vertices of projected mesh, sum(V_i) x 3
        """
        src_verts = y.view(-1,3) # (sum(V_i), 3)
        tgt_verts = self.tgt.verts_packed().detach() # (sum(V_i), 3)
        sqr_diffs = torch.square(src_verts - tgt_verts) # (sum(V_i), 3)

        n_batches = len(self.src)
        sse = torch.zeros(n_batches, dtype=sqr_diffs.dtype)
        if scatter_add:
            sse.scatter_add_(0, self.src.verts_packed_to_mesh_idx(), sqr_diffs)
        else:
            n_verts_per_mesh = self.src.num_verts_per_mesh()
            for i in range(n_batches):
                mesh_to_vert = self.src.mesh_to_verts_packed_first_idx()  # Index of first face per mesh
                start = mesh_to_vert[i]
                end = start + n_verts_per_mesh[i]
                sse[i] = sqr_diffs[start:end].sum()  # Sum over all faces
        return sse

    def equality_constraints(self, xs, y, scatter_add=True):
        """
        Enforces volume constraint
        Assumes same number of vertices in each projected mesh currently

        Args:
            xs (tensor): vertices of original mesh, sum(V_i) x 3
            y (tensor): vertices of projected mesh, sum(V_i) x 3
        """
        n_batches = len(self.src)
        verts_packed = y.view(-1,3) # (sum(V_i), 3)

        faces_packed = self.src.faces_packed()  # (sum(F_i), 3)
        face_vertices = verts_packed[faces_packed]  # (sum(F_i), 3, 3)
        
        # Calculate tetrahedron volumes for each face
        v0, v1, v2 = face_vertices[:, 0, :], face_vertices[:, 1, :], face_vertices[:, 2, :]
        cross_product = torch.cross(v0, v1, dim=-1)  # (F, 3)
        face_volumes = torch.sum(cross_product * v2, dim=-1) / 6.0  # (F,)
        volumes = torch.zeros(n_batches, device=verts_packed.device, dtype=face_volumes.
                                dtype)
        if scatter_add:
            volumes.scatter_add_(0, self.src.faces_packed_to_mesh_idx(), face_volumes)
        else:
            n_faces_per_mesh = self.src.num_faces_per_mesh()
            for i in range(n_batches):
                mesh_to_face = self.src.mesh_to_faces_packed_first_idx()  # Index of first face per mesh
                start = mesh_to_face[i]
                end = start + n_faces_per_mesh[i]
                volumes[i] = face_volumes[start:end].sum()  # Sum over all faces

        volumes = volumes.abs()
        return volumes  # Shape: (B,)    
    
    def solve(self, xs: torch.Tensor):
        n_batches = len(self.src)
        start_vtx = self.src.mesh_to_verts_packed_first_idx()
        end_vtx = start_vtx + self.src.num_verts_per_mesh()
        
        n_vtx = len(self.src.verts_packed())
        results = torch.zeros(n_vtx, 3, dtype=torch.double)
        for batch in range(n_batches):
            start,end = start_vtx[batch],end_vtx[batch]
            verts = xs[start:end].flatten().detach().double().cpu().numpy()
            faces = self.src[batch].faces_packed().detach().double().cpu().numpy()
            tgt_vtx = self.tgt[batch].verts_packed().detach()
            tgt_faces = self.tgt[batch].faces_packed().detach()
            with torch.no_grad():
                tgt_vol = volume_constraint(tgt_vtx, tgt_faces)

            eq_constraint = {
                'type': 'eq',
                'fun' : lambda u: volume_constraint(u, faces, tgt_vol).cpu().numpy(),
                'jac' : lambda u: volume_constraint_grad(u, faces).cpu().numpy()
            }

            res = opt.minimize(
                lambda u: least_squares(u, tgt_vtx),
                verts,
                method='SLSQP',
                jac=lambda u: least_squares_grad(u, tgt_vtx),
                constraints=[eq_constraint],
                options={'ftol': 1e-6, 'iprint': 2, 'maxiter': 100}
            )

            if not res.success:
                print("FAILED:", res.message)
            results[start:end, :] = torch.tensor(res.x, dtype=torch.double, requires_grad=True).view(-1,3)
        return results,None

Pseudo code:
- load in the meshes
- inner problem needs access to the vertices, number of meshes, faces, and indexing
- outer problem needs access to projected vertices, number of meshes, and indexing. Also needs projection matrices, and edge maps of renders, so perform edge detection of renders beforehand.

just provide both with the meshes lol

Projection:
Get the indexing correct for the vertices, take the projection of these vertices

In [5]:
import cv2
from cv2.typing import MatLike

# Apply Canny edge detection
def canny_edge_map(img: MatLike):
    # convert to grayscale
    img_greyscale = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # apply edge detection
    edge_map = cv2.Canny(img_greyscale, threshold1=50, threshold2=250)
    # return edge map
    return edge_map

def get_edgemaps(renders):    
    edgemaps = {k: list(map(canny_edge_map,v)) for k,v in renders.items()}
    return edgemaps

In [6]:
from utils import load_renders, load_camera_matrices

renders_path = "../../../Blender/renders/"
renders = load_renders(renders_path)
edgemaps = get_edgemaps(renders)


import os
import re
from collections import defaultdict
import cv2

def load_camera_matrices(path):
    cameras = defaultdict(dict)
    file_pattern = re.compile(r"^(.*)_(K|RT|P)\.npy$")
    for filename in os.listdir(path):
        match = file_pattern.match(filename)
        if match:
            print(filename)
            cam_name, matrix_type = match.groups()
            filepath = os.path.join(path, filename)
            cameras[cam_name][matrix_type] = torch.tensor(np.load(filepath), dtype=torch.double)
    return cameras

matrices_path = "../../../Blender/cameras"
matrices = load_camera_matrices(matrices_path)

Camera3_RT.npy
Camera0_K.npy
Camera1_RT.npy
Camera2_K.npy
Camera3_K.npy
Camera1_K.npy
Camera1_P.npy
Camera3_P.npy
Camera2_RT.npy
Camera0_RT.npy
Camera2_P.npy
Camera0_P.npy


In [7]:
# outer problem
def create_padded_tensor(vertices, vert2mesh, max_V, B):
    padded = torch.zeros((B, max_V, 3),device=vertices.device)
    for i in range(B):
        mesh_vertices = vertices[vert2mesh == i]
        num_vertices = mesh_vertices.shape[0]
        padded[i, :num_vertices, :] = mesh_vertices
    return padded

class PyTorchChamferLoss(nn.Module):
    def __init__(self, src: Meshes, tgt: Meshes, projmatrices, edgemaps):
        super().__init__()
        self.src = src
        self.tgt = tgt
        self.projmatrices = projmatrices
        self.edgemaps = edgemaps
    
    def project_vertices(self, vertices):
        """
        Projects a set of vertices into multiple views using different projection matrices.

        Args:
            vertices: Tensor of shape (N, 3), representing 3D vertex positions.

        Returns:
            Tensor of shape (P, N, 2), containing projected 2D points in each view.
        """
        V = vertices.shape[0]
        projection_matrices = self.projmatrices

        ones = torch.ones((V, 1), dtype=vertices.dtype, device=vertices.device)
        vertices_homogeneous = torch.cat([vertices, ones], dim=1)  # Shape: (V, 4)

        # Perform batched matrix multiplication (P, 3, 4) @ (V, 4, 1) -> (P, V, 3)
        projected = torch.einsum("pij,vj->pvi", projection_matrices, vertices_homogeneous)  # (P, V, 3)
        
        projected_cartesian = projected[:, :, :2] / projected[:, :, 2:3]  # (P, V, 2)

        return projected_cartesian

    def get_boundary(self, projected_pts, alpha=10.0):
        shaper = Alpha_Shaper(projected_pts)
        alpha_shape = shaper.get_shape(alpha)
        boundary = torch.tensor(alpha_shape.exterior.coords.xy)
        boundary_pts = projected_pts[
            torch.any(torch.isclose(projected_pts[:, None], boundary.T, atol=1e-6).all(dim=-1), dim=1)
        ]
        return boundary_pts

    def forward(self, y):
        # y Shape: (sum Vi, 3) -> reshape nicely into (B, maxV, 3)
        B, max_V = len(self.src), self.src.num_verts_per_mesh().max().item()
        vertices = create_padded_tensor(y, self.src.verts_packed_to_mesh_idx(), max_V, B) # (B, maxV, 3)

        # project vertices
        projected_vertices = [] # (B, P, V, 2)
        for b in range(B):
            projverts = self.project_vertices(vertices[b])  # Shape: (P, V, 2)
            projected_vertices.append(projverts)  # Store without padding

        # get boundaries
        boundaries = [] 
        for batch in projected_vertices:
            boundaries_b = []
            for projverts in batch:
                boundary = self.get_boundary(projverts)
                boundaries_b.append(boundary)
            stacked_boundaries = torch.stack(boundaries_b)
            # padded_boundaries = torch.nn.utils.rnn.pad_sequence(boundaries_b, batch_first=True, padding_value=0.0)
            boundaries.append(stacked_boundaries)

        # perform chamfer
        chamfer_loss = torch.zeros(B)
        for b in range(B):
            boundaries_b = boundaries[b]
            edgemaps_b = self.edgemaps[b]
            res, _ = chamfer_distance(x=boundaries_b.float(),
                                                y=edgemaps_b.float(),batch_reduction="mean",point_reduction="mean")
            chamfer_loss[b] = res.sum()
        return chamfer_loss.double()


In [8]:
import torch
import numpy as np
import matplotlib.pyplot as plt

def project_vertices(vertices, projection_matrix):
    ones = torch.ones((vertices.shape[0], 1), dtype=vertices.dtype, device=vertices.device)
    vertices_homogeneous = torch.cat([vertices, ones], dim=1)  
    
    projected = projection_matrix @ vertices_homogeneous.T  
    
    projected = projected.T  # Shape becomes Nx3
    projected_cartesian = projected[:, :2] / projected[:, 2:3] 
    
    return projected_cartesian



# plt.figure(figsize=(9.6, 5.4)) 
# plt.scatter(points_2d[:, 0], points_2d[:, 1], s=1, c='blue', alpha=0.5)
# plt.xlim(0, 1920)
# plt.ylim(0, 1080)
# plt.gca().invert_yaxis()  # Invert y-axis to match image coordinates (0 at top)

# plt.title('Projected Vertices')
# plt.xlabel('X (pixels)')
# plt.ylabel('Y (pixels)')
# plt.grid(True, alpha=0.3)

# plt.tight_layout()
# plt.savefig('projected_vertices.png', dpi=100)
# plt.show()

In [9]:

paths = [os.path.join("../../../Blender/", f"{name}_2.obj") for name in ["sphere", "balloon", "parabola", "rstrawberry"]]
sphere, balloon, parabola, rstrawberry = load_objs_as_meshes(paths)

P = matrices["Camera0"]["P"]  # Shape is 3x4
vertices = parabola.verts_packed().double()  # Shape is Nx3
points_2d = project_vertices(vertices, P)

In [11]:
alpha = 10.0
shaper = Alpha_Shaper(points_2d)
# alpha_opt, alpha_shape = shaper.optimize()
alpha_shape = shaper.get_shape(alpha=alpha)
# print(torch.tensor(alpha_shape.exterior.coords.xy))

# print(points_2d[:, None, :].size(), points_2d.size())

boundary = torch.tensor(alpha_shape.exterior.coords.xy, dtype=points_2d.dtype)
boundary_pts = points_2d[
    torch.any(torch.isclose(points_2d[:, None], boundary.T, atol=1e-6).all(dim=-1), dim=1)
]
# print(boundary_pts)

sorted_boundary_pts = boundary_pts[boundary_pts[:, 0].argsort()]

# For the exterior coordinates, convert to numpy array first
exterior_coords = np.array(alpha_shape.exterior.coords.xy).T  # Transpose to get (n_points, 2)
sorted_exterior_coords = exterior_coords[exterior_coords[:, 0].argsort()]

print("Sorted boundary points:")
print(sorted_boundary_pts)
print("\nSorted exterior coordinates:")
print(sorted_exterior_coords)

# fig, (ax0, ax1) = plt.subplots(1, 2)
# ax0.scatter(*zip(*points_2d))
# ax0.set_title('data')
# ax1.scatter(*zip(*points_2d))
# ax0.invert_yaxis()
# ax1.invert_yaxis()
# plot_alpha_shape(ax1, alpha_shape)
# ax1.set_title(f"$\\alpha={alpha:.3}$")

# for ax in (ax0, ax1):
#     ax.set_aspect('equal')

# for v in points_2d:
#     if tuple(v) == (620.6065372882819, 542.3838993304273):
#         print(v)

Sorted boundary points:
tensor([[ 620.6065,  542.3839],
        [ 645.6539,  400.9917],
        [ 646.5989,  497.4448],
        [ 652.8959,  606.0431],
        [ 660.6537,  663.5051],
        [ 714.6081,  293.5973],
        [ 725.3716,  320.6722],
        [ 732.1652,  694.8917],
        [ 741.6011,  706.7673],
        [ 797.1500,  250.9754],
        [ 810.4565,  712.7552],
        [ 820.5273,  221.8992],
        [ 873.6121,  728.9413],
        [ 888.9455,  222.3212],
        [ 903.5753,  756.9889],
        [ 915.0647,  810.5560],
        [ 918.4108,  228.9707],
        [ 956.4047,  830.6323],
        [ 959.7287,  244.2515],
        [ 995.0637,  281.5897],
        [1001.4562,  849.5047],
        [1006.5479,  320.6073],
        [1028.8410,  835.9146],
        [1035.5187,  344.4571],
        [1096.9772,  832.4019],
        [1110.5111,  331.4659],
        [1113.7154,  791.3713],
        [1128.1780,  310.4889],
        [1176.4475,  718.1627],
        [1192.5360,  749.2712],
        [1192.66

In [58]:
import torch

vertices = torch.tensor([
    [0.1, 0.2, 0.3], [0.4, 0.5, 0.6], [0.7, 0.8, 0.9],
    [1.0, 1.1, 1.2], [1.3, 1.4, 1.5],
    [1.6, 1.7, 1.8], [1.9, 2.0, 2.1],
], dtype=torch.float32, requires_grad=True)

mesh_data = torch.tensor([0, 0, 0, 1, 1, 2, 2])
B, max_V = 3, 3

def create_padded_tensor(vertices, mesh_data, max_V, B):
    padded = torch.zeros((B, max_V, 3), device=vertices.device)
    
    for i in range(B):
        mask = (mesh_data == i)
        mesh_vertices = vertices[mask]
        num_vertices = mesh_vertices.shape[0]
        padded[i, :num_vertices, :] = mesh_vertices
    
    return padded

# Test gradient flow
padded = create_padded_tensor(vertices, mesh_data, max_V, B)
loss = padded.sum()
loss.backward()

print("Gradient exists:", vertices.grad is not None)
print("Gradient:")
print(vertices.grad)

Gradient exists: True
Gradient:
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
